# Bytewax

In [1]:
from datetime import timedelta

import bytewax.operators as op
from bytewax.dataflow import Dataflow
from bytewax.connectors.stdio import StdOutSink
from bytewax.testing import TestingSource

In [18]:
flow = Dataflow("collect")

In [19]:
step1 = op.input("input", flow, TestingSource(list(range(10))))

In [20]:
step2 = op.output('output', step1, StdOutSink())

# Alpaca

In [1]:
import requests
from pprint import pprint
from websocket import create_connection
import json

In [2]:
API_KEY = 'PKF3KMYDUEAUK0S112HX'
SECRET = 'n1hrmo48mJpBoyV7LtVtx6XflUjZymEwd9d9RFev'
ws_url = "wss://stream.data.alpaca.markets/v1beta1/news"

## Batch

In [10]:
import requests

url = "https://data.alpaca.markets/v1beta1/news"

headers = {
    "accept": "application/json",
    "APCA-API-KEY-ID": API_KEY,
    "APCA-API-SECRET-KEY": SECRET
}

params = {'start': '2024-06-27', 'end': '2024-03-29', 'symbols': ['CRM'], 'include_content': True, 'limit': 50}
response = requests.get(url, headers=headers, params=params)

print(response.text)

{"message":"Start should not be before end"}



In [5]:
import requests

url = "https://data.alpaca.markets/v1beta1/news?start=2024-02-01&end=2024-02-10&sort=desc&symbols=CRM&include_content=true&exclude_contentless=true"

headers = {
    "accept": "application/json",
    "APCA-API-KEY-ID": API_KEY,
    "APCA-API-SECRET-KEY": SECRET
}

response = requests.get(url, headers=headers)

print(response.text)

{"news":[{"author":"Benzinga Insights","content":"\u003cp\u003eInvestors with a lot of money to spend have taken a bullish stance on \u003cstrong\u003eSalesforce\u003c/strong\u003e (NYSE:\u003ca class=\"ticker\" href=\"https://www.benzinga.com/stock/CRM#NYSE\"\u003eCRM\u003c/a\u003e).\u003c/p\u003e\n\u003cp\u003eAnd retail traders should know.\u003c/p\u003e\n\u003cp\u003eWe noticed this today when the trades showed up on publicly available options history that we track here at Benzinga.\u003c/p\u003e\n\u003cp\u003eWhether these are institutions or just wealthy individuals, we don't know. But when something this big happens with CRM, it often means somebody knows something is about to happen.\u003c/p\u003e\n\u003cp\u003eSo how do we know what these investors just did? \u003c/p\u003e\n\u003cp\u003eToday, \u003ca href=\"https://pro.benzinga.com/?utm_source=insights\"\u003eBenzinga\u003c/a\u003e's options scanner spotted 42 uncommon options trades for Salesforce.\u003c/p\u003e\n\u003cp\u00

## Stream (but didnt work or keeps running)

In [3]:
conn = create_connection(ws_url)
conn.recv()

'[{"T":"success","msg":"connected"}]'

In [4]:
conn.send(
        json.dumps(
            {
                "action": "auth",
                "key": API_KEY,
                "secret": SECRET,
            }
        )
    )

conn.recv()

'[{"T":"success","msg":"authenticated"}]'

In [5]:
conn.send(
            json.dumps({"action": "subscribe", "news": ['GOOG']})
        )


44

In [7]:
conn.recv()

# Bytewax and alpaca

In [1]:
from datetime import timedelta

import bytewax.operators as op
from bytewax.dataflow import Dataflow
from bytewax.connectors.stdio import StdOutSink
from bytewax.inputs import DynamicSource
from bytewax.outputs import DynamicSink
import requests
import json

In [2]:
API_KEY = 'PKF3KMYDUEAUK0S112HX'
SECRET = 'n1hrmo48mJpBoyV7LtVtx6XflUjZymEwd9d9RFev'

In [3]:
# util
from datetime import datetime, timedelta

def split_date_range(from_date, to_date, num_intervals):
    # Convert input strings to datetime objects
    from_date = datetime.strptime(from_date, '%Y-%m-%d')
    to_date = datetime.strptime(to_date, '%Y-%m-%d')
    
    # Calculate total number of days between from_date and to_date
    total_days = (to_date - from_date).days
    
    # Calculate the number of days per interval
    days_per_interval = total_days / num_intervals
    
    # Initialize list to store result tuples
    result = []
    
    # Generate intervals
    for i in range(num_intervals):
        # Calculate start and end dates for the current interval
        start_date = from_date + timedelta(days=int(i * days_per_interval))
        end_date = from_date + timedelta(days=int((i + 1) * days_per_interval))
        
        # Append tuple (start_date, end_date) to the result list
        result.append((start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')))
    
    return result


In [8]:
class AlpacaBatch(DynamicSource):
    def __init__(self, from_date, to_date, tickers):
        self.from_date = from_date
        self.to_date = to_date
        self.tickers = tickers 
        self.api_key = API_KEY
        self.secret = SECRET
        # to change the string values
        self.url = "https://data.alpaca.markets/v1beta1/news?start=2024-02-01&end=2024-02-10&sort=desc&symbols=CRM&include_content=true&exclude_contentless=true"
    
    def build(self, worker_index, worker_count):
        print('inside build')
        worker_date_range = split_date_range(self.from_date, self.to_date, worker_count)
        current_from_date, current_to_date = worker_date_range[worker_index] # current wormer's value
        return self.build_alpaca_client(current_from_date, current_to_date)

    def build_alpaca_client(self, current_from_date, current_to_date):
         
        headers = {
                        "accept": "application/json",
                        "APCA-API-KEY-ID": self.api_key,
                        "APCA-API-SECRET-KEY": self.secret
                    }
        params = {
            "start": current_from_date,
            "end"  : current_to_date,
            "sort" : "desc",
            "symbols": self.tickers,
            "include_content": True,
            "page_token": 50,
            "limit": 50
        }
    
        response = requests.get(self.url, headers=headers, params= params)

        next_page_token = None
        if response.status_code == 200:  # Check if the request was successful
            # parse response into json
            news_json = response.json()

            # extract next page token (if any)
            next_page_token = news_json.get("next_page_token", None)

        else:
            print(f"Request failed with status code: {response.status_code}")

        self._page_token = next_page_token

        return news_json["news"]


In [9]:
def build():
    flow = Dataflow("alpaca_news")
    alpaca_batch = op.input("input", flow, AlpacaBatch)
    return alpaca_batch

In [10]:
mynews = build()

In [7]:
mynews

Stream(stream_id='alpaca_news.input.down', _scope=_Scope(parent_id='alpaca_news'))